In [1]:
import numpy as np
import pandas as pd
%matplotlib qt
import sys
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=RuntimeWarning)
sys.path.append('../../')
import Utils.Preprocess as ut
project_path = os.path.abspath(os.path.relpath('../../../../', os.getcwd()))
chb_root_path=os.path.join(project_path,'Dataset/CHB-MIT')

constrain_path=os.path.join(project_path,'BilinearNetwork/Data/Constraint/Prediction')
seizure_table=pd.read_csv(os.path.join(constrain_path,'seizure_preictal_summary.csv'))

exclude_patient=pd.read_csv(os.path.join(project_path,'BilinearNetwork/Data/Constraint/Prediction/exclude_Patient.csv'))

save_path=os.path.join(project_path,'BilinearNetwork/Data/PreprocessedData/CHB-MIT/Prediction')

In [2]:
def extract_epochs(raw_path,start,stop):
    raw_T=ut.PreprocessTool(raw_path,duration=5)
    raw_T.do_preprocess().overlap_events(start,stop).cut_epochs()
    epochs_data=raw_T.get_epochs().get_data().astype(np.float32)
    stft_data=raw_T.get_epochs_stft().astype(np.float32)
    del raw_T
    return epochs_data,stft_data

In [3]:
for row in tqdm(seizure_table.iterrows(),total=seizure_table.shape[0]):
    file_name=row[1]['File Name']
    prelix=file_name.split('_')[0]
    chb_patient_path=os.path.join(chb_root_path,prelix.rstrip('a').rstrip('b'))
    process_file_path=os.path.join(chb_patient_path,file_name)
    start=row[1]['Preictal Start Time']
    stop=row[1]['Preictal End Time']

    if prelix in list(exclude_patient['0']):
        continue
    if stop-start<5:
        continue
    try:
        _,local_epoch_stft=extract_epochs(process_file_path,start,stop)
    except ValueError as e:
        print(e)
        break
    assert local_epoch_stft.shape[0]>0
    
    np.save(os.path.join(save_path,'{}_preictal.npy'.format(file_name.split('.')[0])),local_epoch_stft)
    
    
    

In [ ]:
# file_name='chb09_08.edf'
# prelix=file_name.split('_')[0]
# chb_patient_path=os.path.join(chb_root_path,prelix.rstrip('a').rstrip('b'))
# process_file_path=os.path.join(chb_patient_path,file_name)
# start=7396
# stop=8896
# 
# 
# _,local_epoch_stft=extract_epochs(process_file_path,start,stop)
# 
# np.save(os.path.join(save_path,'chb09_08(1)_preictal.npy'),local_epoch_stft)